# The Impact of state utility industries' Gross State Product(GSP) on the Severity of its Power Outage

**Name(s)**: Guoxuan Xu, Qirui Zheng

**Website Link**: [https://qz07.github.io/powerOutage-Analysis/](https://qz07.github.io/powerOutage-Analysis/)

## Code

In [123]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

from scipy.stats import ks_2samp

### Introduction

While it is important for each state to consider the emergency resolutions to Major Power outage within state, something that is more important is that state should find main factors that could constribute to sever in-state power outage so that the catastrophe can be prevent before even causing any damages.

Here are some possibe data science questions we can explore regarding the main factors that contributed to sever power outage:
- Is there a postive correlation between the industrial customer population and duration of power outage?
- Is the power outage duration mainly depends on the climate and geographical condition in different areas?
- Will the increasing in population density cause the increasing in power outage duration since the it will take them more time to get more electricity
- Is sever power outage happen at certain time frame of the day across America?

The **central data science question** for this study is <u>whether the state utility industries' Gross State Product(GSP) is a significant factor that contributes to the severity of its major power outages?</u> We originally plan to explore the impact of overall state GSP on its power outages, however, after EDA, we decide to change the question to current one. Beside, intuitively, it makes more sense to focus on the GSP produced by the utility industries because they are the main stackholder that control the supply of electricty of the state. 

Through out the project, we are planning to draw possible correlations between the data about the state utility industries with the power outage duration in minutes.

### Getting the data 


The dataset from, Laboratory for Advancing Sustainable Critical Indrastructure, named: Major POwer OUtage Risks in teh US, by Purdue University is downloaded in excel format. However, when reading the file in can cause errors due to the structure of the dataset. To be able to read in the correct structure of the dataset we took out unecessay rows that <u>does not below to the data set</u> and saved the true data in a csv format, that is later read in below in a dataset called `powerOutage`.

In [124]:
powerOutage = pd.read_csv('data/outageClean.csv')
powerOutage

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
1,2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,"Tuesday, December 6, 2011",...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256
1532,1533,2009,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,"Saturday, August 29, 2009",...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256


### Cleaning and EDA

#### Data cleaning

***
cleaning overview:
- converting time related to timestamp object
- filter out unnecessary columns
- find proportion of utility industry GSP contribution of each state
- add boolean tags to each row on whether the state has relative GSP that is higher than the average US GSP of that specific year
- add boolean tags to each row on whether the state has relative utility GSP that is higher than the average US utility GSP of that specific year
***

Step 1: converting time related column to timestamp object

In [125]:
#Take two columns one for date and one for time, combine both columns and convert to timestamp object 

def clean_time(df, col1, col2, name):
    #col1 is a date use pd.to_datetime, string
    #col2 is a time use pd.to_timedelta, string 
    changed = pd.to_datetime(df[col1], format = '%A, %B %d, %Y') + pd.to_timedelta(df[col2])
    return df.assign(**{name: changed}).drop(columns = [col1, col2])

Step 2: pick the columns that contain general power outage information and columns that relate to economic performance and power outage severity of each state. 

In [126]:
#Take in a dataframe and return only the columns that we need

def filter_columns(df, *dropRanges):
    for dropRange in dropRanges:
        if dropRange[0] == None:
            df = df.drop(columns=df.loc[:, :dropRange[1]])
        elif dropRange[1] == None:
            df = df.drop(columns=df.loc[:, dropRange[0]:])
        else:
            df = df.drop(columns=df.loc[:, dropRange[0]:dropRange[1]])
    return df

Step 3: Aside from the overall GSP production of each state, we also want to explore proportion of GSP contribute by states' utility industries from its overall GSP. We will defined new column `UTIL.REALGSP.CONTRI.PROP` by dividing `UTIL.REALGSP` column by `TOTAL.REALGSP`

In [127]:
#Take in a dataframe and proportion of contribution made by utility industry into the dataframe

def add_util_prop(df):
    return df.assign(**{'UTIL.REALGSP.CONTRI.PROP':(df['UTIL.REALGSP']/df['TOTAL.REALGSP'])})

Step 4: identify whether the state of each major power outage(each row) has relative GSP that is higher than the average US GSP of that specific year. We will define new column `stateGSPaboveAvg` by checking if `PC.REALGSP.REL` is greater or equal to 1



In [128]:
# in the stateGSPaboveAvg, True means the state overall GSP performance is above average, 
# while False means the state overall GSP performance is belong average

def check_wGSP(df):
    return df.assign(stateGSPaboveAvg = df['PC.REALGSP.REL'] >= 1)

Step 5: identify whether the state of each major power outage(each row) has relative __utility__ GSP that is higher than the average utility GSP of that specific year. We will define new column `untilGSPaboveAvg` by checking if `UTIL.REALGSP.CONTRI.PROP` is greater or equal to the average utility GSP of that year


In [129]:
# in the untilGSPaboveAvg, True means the state utility GSP performance is above average, 
# while False means the state utility GSP performance is belong average

def check_uGSP(df):
    return df.assign(untilGSPaboveAvg = df.groupby('YEAR')['UTIL.REALGSP.CONTRI.PROP'].transform(lambda series : series >= series.mean()))

Chaining all the cleaning method on the original dataset

In [130]:
powerOutage = (
    pd.read_csv('data/outageClean.csv')
    .pipe(clean_time, col1 = 'OUTAGE.START.DATE', col2 = 'OUTAGE.START.TIME',name = 'OUTAGE.START')
    .pipe(clean_time, col1 = 'OUTAGE.RESTORATION.DATE', col2 = 'OUTAGE.RESTORATION.TIME', name = 'OUTAGE.RESTORATION')
    .pipe(filter_columns, ('CLIMATE.REGION', 'HURRICANE.NAMES'), ('RES.CUSTOMERS', 'IND.CUST.PCT'), ('AREAPCT_URBAN', 'PCT_WATER_INLAND'))
    .pipe(add_util_prop)
    .pipe(check_wGSP)
    .pipe(check_uGSP)
)

Extra note:
We notice that some value in outage durations are 0 which can potentially be missing value, however, considering the fact that we durations were calculated in minutes. There is possibility that the duration is under 1 minute but were being recorded in 0 minutes. Therefore, we decide not to change 0 time duration to NaN.

In [131]:
# Cleaned dataframe 
powerOutage.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,OUTAGE.START,OUTAGE.RESTORATION,UTIL.REALGSP.CONTRI.PROP,stateGSPaboveAvg,untilGSPaboveAvg
0,1,2011,7.0,Minnesota,MN,MRO,3060.0,NaN,70000.0,11.60,...,73.27,15.28,2279.0,1700.5,18.2,2011-07-01 05:00:00,2011-07-03 08:00:00,0.017514,True,False
1,2,2014,5.0,Minnesota,MN,MRO,1.0,NaN,NaN,12.12,...,73.27,15.28,2279.0,1700.5,18.2,2014-05-11 06:38:00,2014-05-11 06:39:00,0.017900,True,True
2,3,2010,10.0,Minnesota,MN,MRO,3000.0,NaN,70000.0,10.87,...,73.27,15.28,2279.0,1700.5,18.2,2010-10-26 08:00:00,2010-10-28 10:00:00,0.017063,True,False
3,4,2012,6.0,Minnesota,MN,MRO,2550.0,NaN,68200.0,11.79,...,73.27,15.28,2279.0,1700.5,18.2,2012-06-19 04:30:00,2012-06-20 11:00:00,0.019321,True,True
4,5,2015,7.0,Minnesota,MN,MRO,1740.0,250.0,250000.0,13.07,...,73.27,15.28,2279.0,1700.5,18.2,2015-07-18 02:00:00,2015-07-19 07:00:00,0.016687,True,True


#### Exploratory Data Analysis

***
EDA overview:
- distribution of power outage duration in minutes(univariate graph on: `OUTAGE.DURATION`)
- relationship between year and the power outage duration(bivariate graph on: `YEAR` and `OUTAGE.DURATION`)
- distribution of relative real GSP of each states with major power outage across different time period(univariate graph on `PC.REALGSP.REL`)
- distribution of the proportion of utility GSP constribution of each state across different time period(univariate graph on `UTIL.REALGSP.CONTRI.PROP`)
- relationship between year and the states' average real GSP(bivariate grah on `YEAR` and `PC.REALGSP.REL`)
- median power duration with high/bad relative state GSP and high/bad relative utility GSP (pivote table on `PC.REALGSP.REL`, `UTIL.REALGSP.CONTRI.PROP`, and `OUTAGE.DURATION`)
***


Distribution of power outage duration in minutes across all the time (__Univariate__)

In [132]:
#Is there any relationship of duration against itself? or is there any trend?

px.histogram(powerOutage, x='OUTAGE.DURATION',
             title="Distribution of power outage duration in minutes",
             histnorm='probability density',
             labels={'OUTAGE.DURATION': "Duration of Outage (in mins)"})

Possible conclusion on above graph:
- It is a right skewed graph, so that means majority power outage is within 20_000 mins
- the power outage that last particularly long could be possible outliers
- using <u>median</u> will be more accurate in measuring the central tendency of the outage duration

Relationship between year and power outage duration (__Bivariate__)

In [133]:
# As time pass by from year to year, does the median power outage decreased or increased?

px.scatter(powerOutage.groupby('YEAR')['OUTAGE.DURATION'].median(),
           title='Year vs. Power Outage Duration in minutes', 
           labels={'value': 'Outage Duration (mins)'})         

Possible conclusion on above graph:
- there is a weak negative liner correlation between year and median power outage. The power outage becomes shorter and shorter
- Is this <u>decreasing due to the change of GSP</u> each year?
- There is a signifcant fluctuation from 2000 to 2002

Since we talk about the possible imapcts of GSP on the power outage duration, let's look at the distribution of relative GSP of each state (__Univariate__)

In [134]:
# What is the tendency of the GSP power outage

fig1 = px.histogram(powerOutage, x="PC.REALGSP.REL",
             title="The distribution of relatived GSP among states",
             histnorm='probability density',
             labels={'PC.REALGSP.REL': 'Relative GSP of each state compare to yearly US average GSP'})

fig1.show()


Possible conclusion on above graph:
- The distribution of GSP in the graph is multimodel(2 peaks) which indicates that there might be multiple <u>'clusters' or groups</u> where state tend to have their GSP relative to the US average 

Following that pattern of thoughts, we can try to find the distribution of relative utility GSP across different states (__Univariate__)

In [135]:
# What is the tendency of the relative uility GSP power outage

px.histogram(powerOutage, x="UTIL.REALGSP.CONTRI.PROP",
             title="The distribution of relatived uility GSP among states",
             histnorm='probability density',
             labels={'UTIL.REALGSP.CONTRI.PROP': 'relative utility GSP of each state compare to yearly US average utility GSP'})

Possible conclusion on above graph:
- Again! There is similar trend compared to the distribution of relative state GSP in previous graph 

Is there any relationship between time and the the relative utility GSP? (__Bivariate__)

In [136]:
# As time pass by from year to year, does the mean relative utility GSP

px.scatter(powerOutage.groupby('YEAR')['UTIL.REALGSP.CONTRI.PROP'].mean(),
           title='Year vs. mean relative GSP contributed by utility industry', 
           labels={'value': 'utlity industries\' relative GSP'})      

Possible conclusion on above graph:
- Apperantly, there is a decreasing trend on the utility GSP contribution over times
- It correponds to the previous line plot on the year vs. outage time duration. This implies that as the time go <u>both outage time duration and relative GSP constribution of utility industry decrease</u>

Distribution of median power duration with high/low relative state GSP and high/low relative utility GSP (__pivote table__)

In [137]:
# which category does the medians powerOutage got minimized

powerOutage.pivot_table(index= 'stateGSPaboveAvg',
                                 columns = 'untilGSPaboveAvg',
                                 values = 'OUTAGE.DURATION',
                                 aggfunc = 'median')

untilGSPaboveAvg,False,True
stateGSPaboveAvg,,
False,272.0,1440.0
True,369.5,870.0


***

Base on the table above, suprisingly it seems like <u>the state with low relative utility GSP has the least power outage duration</u>. Furthermore, the overall outage duration of states with lower relative utility GSP(`untilGSPaboveAvg` is False) tend to be lower than the states with high relative utility GSP(`untilGSPaboveAvg` is True). Based of that, we hypothesize that the states with large proportion of GSP utility constribution are more likely to have large utility base. If they experience power outage, it might takes the large utility sectors more times to restore the outage. However, we need more domain knowledge to assest the claim. This is a really interesting trend for us to explore to explore to later hypothesis test!

***

### Assessment of Missingness

Our data set is comprehensive in terms of the overall missingness. However, one of the most important column to assest outage severity `OUTAGE.DURATION` has non-trival missing values. It is critical that its missingness related to other columns, so we can perform valid imputation on the those missing variable. Once we shuffle the data set and the p value is less than significance level(0.05), then it means that the columns are dependent on each other

#### MISSINGNESS TEST #1: Possible correlation between `POPULATION` and missingness of `OUTAGE_DURATION`

Intuitively, we hypothesis that the missingness of power outage duration might depends on the populations. The population decide the amount of people that will be impacted by the power outage. If population of the state is relative lower, then less people will be impacted by the power outage. Therefore, they are more likely not to record the outage duration

Before running permutation test, we want to decide the test statistic that will be used to measure the difference between two distributions. If the distributions are similar to each other, we can use the absolute mean difference. If the distribute has different shapes, we can use K-S test statistic which measures the difference between graphs'cumulative distribution.

In [151]:
missing = powerOutage.copy()
missing = missing.assign(duration_miss = missing['OUTAGE.DURATION'].isna())
missing

population_with_missingDuration = powerOutage[powerOutage['OUTAGE.DURATION'].isna()]
px.histogram(population_with_missingDuration, x= 'POPULATION')

population_with_Duration = powerOutage[~powerOutage['OUTAGE.DURATION'].isna()]
px.histogram(population_with_Duration, x= 'POPULATION')

df =pd.DataFrame(dict(
    series=np.concatenate((["population_with_missingDuration"]*len(population_with_missingDuration), ["population_with_Duration"]*len(population_with_Duration))), 
    data  =np.concatenate((population_with_missingDuration['POPULATION'], population_with_Duration['POPULATION']))
))

px.histogram(df,
             x="data",
             color="series",
             barmode="overlay",
             histnorm='probability density',
             labels={'data': 'Population'},
             title='Distribution of Population with and without missing outage duration')

The distribution is different which tell us that we should implement K-S test statistic in our permutation test. The Emprical Distribution of test static below:

In [152]:
n_repetitions = 1_000
shuffled = powerOutage.copy()
shuffled['missing_dur'] = shuffled['OUTAGE.DURATION'].isna()

ks_stats = []
groups = shuffled.groupby('missing_dur')['POPULATION']
obs_stat = ks_2samp(groups.get_group(True), groups.get_group(False)).statistic
for _ in range(n_repetitions):
    
    # Shuffling the data.
    shuffled['POPULATION'] = np.random.permutation(shuffled['RES.PRICE'])
    
    # Computing and storing the K-S statistic.
    groups = shuffled.groupby('missing_dur')['POPULATION']
    ks_stat = ks_2samp(groups.get_group(True), groups.get_group(False)).statistic
    ks_stats.append(ks_stat)
    
ks_stats[:10]

fig = px.histogram(x = ks_stats, 
                   histnorm='probability density',
                   title= 'emprical distribution of K-S test statistic',
                   labels={'x' : 'K-S test statistic'})
fig.add_vline(obs_stat, line_width = 3, line_color = 'red', opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed K-S = {round(obs_stat, 2)}</span>',
                   x=1.1 * obs_stat, showarrow=False, y=12)
fig.show()

In [153]:
p_value = ks_2samp(powerOutage.loc[powerOutage['OUTAGE.DURATION'].isna(), 'POPULATION'], powerOutage.loc[~powerOutage['OUTAGE.DURATION'].isna(), 'POPULATION']).pvalue
p_value

0.35096747985150195

***
Since the p value after running permutation test is 0.35 which is greater than 0.05, we failed to reject the null that the `POPULATION` distribution with missing power outage duration comes from the same population as the `POPULATION` distribution without missing power outage.

The missingness of `OUTAGE_DURATION` is indepdent from the `POPULATION`
***

#### MISSINGNESS TEST #2: Possible correlation between `RES.PRICE` and missingness of `OUTAGE_DURATION`

We also hypothesize that the `RES.PRICE` might impact missingness of the `OUTAGE_DURATION` column. `RES.PRICE` columns inform us the monthly electricty price in the residential sector. We think that the residents in the sector with high monthly electricity price care more about the electricty usage statistic which includes the outage duration so they are more likely to report the outage duration with detailized starting and ending time. On the other hand, residents live in sector with low monthly electricity price might not care about the outage duration since there is no much of differences on their electricity bill if there is an outage.

Just like the first permutation test, we want to define the test statistic accoridng to the distribution of `RES.PRICE`

In [154]:
missing = powerOutage.copy()
missing = missing.assign(duration_miss = missing['OUTAGE.DURATION'].isna())
missing

resPrice_with_missingDuration = powerOutage[powerOutage['OUTAGE.DURATION'].isna()]
px.histogram(resPrice_with_missingDuration, x= 'RES.PRICE')

resPrice_with_Duration = powerOutage[~powerOutage['OUTAGE.DURATION'].isna()]
px.histogram(resPrice_with_Duration, x= 'RES.PRICE')

df =pd.DataFrame(dict(
    series=np.concatenate((["resPrice_with_missingDuration"]*len(resPrice_with_missingDuration), ["resPrice_with_Duration"]*len(resPrice_with_Duration))), 
    data = np.concatenate((resPrice_with_missingDuration['RES.PRICE'], resPrice_with_Duration['RES.PRICE']))
))

px.histogram(df,
             x="data",
             color="series",
             barmode="overlay",
             histnorm='probability density',
             labels={'data': 'Monthly electricity price in residential sector (cents/kilowatt-hour)'},
             title='Distribution of residential monthly electricty cost with and without missing outage duration')

The distribution is different which tell us that we should implement K-S test statistic in our permutation test. The Emprical Distribution of test static below:

In [155]:
n_repetitions = 10_000
shuffled = powerOutage.copy()
shuffled['missing_dur'] = shuffled['OUTAGE.DURATION'].isna()

ks_stats = []
groups = shuffled.groupby('missing_dur')['RES.PRICE']
obs_stat = ks_2samp(groups.get_group(True), groups.get_group(False)).statistic
for _ in range(n_repetitions):
    
    # Shuffling the data.
    shuffled['RES.PRICE'] = np.random.permutation(shuffled['RES.PRICE'])
    
    # Computing and storing the K-S statistic.
    groups = shuffled.groupby('missing_dur')['RES.PRICE']
    ks_stat = ks_2samp(groups.get_group(True), groups.get_group(False)).statistic
    ks_stats.append(ks_stat)
    
fig = px.histogram(x = ks_stats, 
                   histnorm='probability density',
                   title= 'emprical distribution of K-S test statistic',
                   labels={'x' : 'K-S test statistic'})
fig.add_vline(obs_stat, line_width = 3, line_color = 'red', opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed K-S = {round(obs_stat, 4)}</span>',
                   x=1.1 * obs_stat, showarrow=False, y=12)
fig.show()

In [156]:
p_value = ks_2samp(powerOutage.loc[powerOutage['OUTAGE.DURATION'].isna(), 'RES.PRICE'], powerOutage.loc[~powerOutage['OUTAGE.DURATION'].isna(), 'RES.PRICE']).pvalue
p_value

0.008940294475536374

***

After running permutation test, the p value is 0.0089 which is less than the standard significance level 0.05. We reject the null hypothesis and accept the alternative hypothesis that the distributions of `RES.PRICE` with and without missing outage duration is different from each other.

The missingness of `OUTAGE_DURATION` is dependent on the `RES.PRICE` column, this may imply the missingness of `OUTAGE.DURATION` is missing at random (MAR)

***

For future work,
with the conclusion from above, we can impute Outage_duration missing value using the `RES.PRICE` column. Since the existance of outliers in `OUTAGE.DURATION` weights more, the imputation planned is median imputation. 

### Hypothesis Testing

After data cleaning & EDA, careful review on the missingness of one of the important column, we are able to answer the question we mentioned eariler. To answer the qestion: whether the states' relative utility GSP is one of the leading factors that impact the severity of the power outage? We will run permutation test on `OUTAGE_DURATION` column and `UTIL.REALGSP.CONTRI.PROP`. In data cleaning stage, we have already define the goodness(high/low) of each rows' relative utility GSP base on their average each year. In hypothesis test, we will use them to separate time duration into two groups and measure the difference between their distributions

In doing this, our hypothesis will be:
- null hypothesis: Both states with high utility GSP and states with low utility GSP will __likely share the same__ distribution of power outage duration
- alternative hypothesis: States with high utility GSP will have __different__ distribution of power outage duration from the states with low utility GSP

First of all, comparing two distribution to decide the test statistic

In [146]:
good_utility = powerOutage[powerOutage['untilGSPaboveAvg']]
px.histogram(good_utility, x= 'OUTAGE.DURATION')

bad_utility = powerOutage[~powerOutage['untilGSPaboveAvg']]
px.histogram(bad_utility, x= 'OUTAGE.DURATION')

df =pd.DataFrame(dict(
    series=np.concatenate((["high_utility_GSP"]*len(good_utility), ["low_utility_GSP"]*len(bad_utility))), 
    data  =np.concatenate((good_utility['OUTAGE.DURATION'], bad_utility['OUTAGE.DURATION']))
))

px.histogram(df,
             x="data",
             color="series",
             barmode="overlay",
             histnorm='probability density',
             labels={'data': 'Power Outage Duration in minute'},
             title='Distribution of power outage for group with high utility GSP and low unility GSP')

The shape of two distributions look similar to each other, so we can use absolute mean difference to measure the similarity between two distributions.

In [147]:
#permutation function using absolute mean difference as test statistic

def abs_diff_p_val(df, col1, col2, N):
    '''
    return the p-value of a permuatation using absolute mean difference
    df: the dataframe to use 
    col1 the permutated column that has missingness 
    col2 the dependency column 
    N the number of iterations
    '''
    #True when the value is missing, False if the value exist
    df = df [~df[col2].isna()]
    is_col1 = df[col1]
    is_col2 = df[col2].values 
    non_miss = is_col1.sum()
    miss = len(is_col1)-non_miss
    
    obs_no_miss = (is_col2*~is_col1).sum()/non_miss
    obs_miss = (is_col2*is_col1).sum()/miss
    obs_stat = abs(obs_no_miss-obs_miss)
    
    miss_permutation = np.column_stack([
        np.random.permutation(is_col1)
        for _ in range(N)
    ]).T
    
    mean_no_miss = (is_col2*~miss_permutation).sum(axis=1)/non_miss
    mean_miss = (is_col2*miss_permutation).sum(axis=1)/miss
    diff = abs(mean_no_miss-mean_miss)
    

    return np.mean(diff >= obs_stat), diff, obs_stat

p_value, emprical, obs = abs_diff_p_val(powerOutage, 'untilGSPaboveAvg','OUTAGE.DURATION',10_000)

The emprical distribution of test statistic belong:

In [148]:
fig = px.histogram(x = emprical, 
                   histnorm='probability density',
                   title= 'emprical distribution of Observed absolute mean difference',
                   labels={'x' : 'Obsolte mean difference'})
fig.add_vline(obs, line_width = 3, line_color = 'red', opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed absolute mean difference = {round(obs_stat, 4)}</span>',
                   x=0.85*obs, showarrow=False, y=0.001)

fig.show()

In [149]:
p_value

0.0

### Conclusion

since the p value is 0.0 which is less than significance level $\alpha$ = 0.05, we reject the null hypothesis and accpet the alternative hypothesis that the distribution of power outage with high utility GSP is different from the distribution of power outage with bad utility GSP.

We can probably say that the economic performance of a state on its utility industries is a main factor that impact the severity of its major power outages.